In [28]:
import warnings
warnings.filterwarnings('ignore')
import os, base64,io
import sys
import xml.etree.ElementTree as ET
import rdflib
import re
from lxml import etree
import copy
import numpy as np
from numpyencoder import NumpyEncoder
import pandas as pd
import math
import operator as op
import xlsxwriter
import json
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, IntSlider, FloatSlider, Layout, FloatRangeSlider
from IPython.display import display, clear_output, Image, FileLink, HTML
from PIL import Image
from scipy.optimize import curve_fit, least_squares
import requests
from scipy.integrate import solve_ivp, LSODA
import matplotlib.pyplot as plt
from matplotlib import markers
import matplotlib.font_manager as font_manager
import matplotlib.colors
from pylab import rcParams
import matplotlib.image as mpimg
import matplotlib.ticker as ticker
import gzip
import csv


In [32]:
url = "https://data.bioontology.org/ontologies/OPB/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb&download_format=csv"  # URL of the GZ file
filename = "OPB.csv.gz"  # Name of the GZ file to be saved
csv_filename = "OPB.csv"  # Name of the CSV file to be extracted

response = requests.get(url)  # Download the file

if response.status_code == 200:  # Check if the download was successful
    # Open the gzipped file and read the content into memory
    with open(filename, 'wb') as f:
        f.write(response.content)
    # print("File downloaded and saved as:", filename)
    
    # Extract the CSV file from the gzipped file
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        csv_reader = csv.reader(f)
        with open(csv_filename, 'w', newline='', encoding='utf-8') as out_file:
            csv_writer = csv.writer(out_file)
            for row in csv_reader:
                csv_writer.writerow(row)
#     print("CSV file extracted and saved as:", csv_filename)
# else:
#     print("Failed to download the file.")

File downloaded and saved as: OPB.csv.gz
CSV file extracted and saved as: OPB.csv


In [33]:
url = "https://data.bioontology.org/ontologies/CHEBI/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb&download_format=csv"  # URL of the GZ file
filename = "CHEBI.csv.gz"  # Name of the GZ file to be saved
csv_filename = "CHEBI.csv"  # Name of the CSV file to be extracted

response = requests.get(url)  # Download the file

if response.status_code == 200:  # Check if the download was successful
    # Open the gzipped file and read the content into memory
    with open(filename, 'wb') as f:
        f.write(response.content)
    # print("File downloaded and saved as:", filename)
    
    # Extract the CSV file from the gzipped file
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        csv_reader = csv.reader(f)
        with open(csv_filename, 'w', newline='', encoding='utf-8') as out_file:
            csv_writer = csv.writer(out_file)
            for row in csv_reader:
                csv_writer.writerow(row)
#     print("CSV file extracted and saved as:", csv_filename)
# else:
#     print("Failed to download the file.")

File downloaded and saved as: CHEBI.csv.gz


In [34]:
url = "https://data.bioontology.org/ontologies/FMA/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb&download_format=csv"  # URL of the GZ file
filename = "FMA.csv.gz"  # Name of the GZ file to be saved
csv_filename = "FMA.csv"  # Name of the CSV file to be extracted

response = requests.get(url)  # Download the file

if response.status_code == 200:  # Check if the download was successful
    # Open the gzipped file and read the content into memory
    with open(filename, 'wb') as f:
        f.write(response.content)
    # print("File downloaded and saved as:", filename)
    
    # Extract the CSV file from the gzipped file
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        csv_reader = csv.reader(f)
        with open(csv_filename, 'w', newline='', encoding='utf-8') as out_file:
            csv_writer = csv.writer(out_file)
            for row in csv_reader:
                csv_writer.writerow(row)

In [35]:
savedModels=[]
savedAnnotations={}
savedkfkr={}
savedtypes={}
savedSpeciesNoDuplicateSingleDict={}
initialValues={}
Nmatrices={}

In [36]:
def pVals(singleSelection,floatSSget):

    # This section works for 'type 1'
    if singleSelection.value == 'type 1':

        solve = tuple()
        solve = solve+([el.value for el in floatSSget if el.description=='Ai'][0],)
        solve = solve+([el.value for el in floatSSget if el.description=='Ao'][0],)


        bounds=[[],[]]
        for i in range(6):
            bounds[0].append(0)
            bounds[1].append(np.inf)



        def func(X, p1,p2,p3,p4,p5,p6):
            Ai,Ao = X

            V_SS = E_tot.value* (p5* Ao - p6* Ai)/ (p1 + p2*Ao + p3*Ai*Ao + p4*Ai)

            return V_SS


        popt, pcov = curve_fit(func,solve,[el.value for el in floatSSget if el.description=='V_SS'][0], maxfev=3000000, bounds=bounds)
        if all(x==1 for x in popt): 
            text0=widgets.HTML(value="<h3><b><font color='red'>Convergence Error: Low E_tot or zero V_SS.<b><h3>")
            display(text0)


        pVals={}
        for i,el in enumerate(popt):
            pVals['p'+str(i+1)]=el



        json_object = json.dumps(pVals, indent=4, sort_keys=True,
              separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/pVals.json", "w") as outfile:
            outfile.write(json_object)
        
    return


In [37]:
def kfkr_estimation(singleSelection):
    f = open("./temporary files/pVals.json")
    pVals = json.load(f)

    if singleSelection.value == 'type 1':
        # Define the function that returns the difference between 
        # the equations and the known values
        def equations(variables, *data):
            k_1f, k_2f, k_3f, k_4f, k_1r, k_2r, k_3r, k_4r = variables
            p1, p2, p3, p4, p5, p6, p7, p8 = data

            eq1 = k_1f * k_3r * k_4f + k_1f * k_2r * k_3r + k_2r * k_3r * k_4r - p1*k_3f*k_4f
            eq2 = k_1f * k_2f * k_4f + k_2f * k_3f * k_4f + k_1f * k_2f * k_3f - p2*k_3r*k_4r
            eq3 = k_1r * k_2f * k_4f + k_1r * k_2f * k_4r - p3*k_1f
            eq4 = k_1r * k_2r * k_4r + k_1r * k_3r * k_4f + k_1r * k_2r * k_3r + k_1r * k_3r * k_4r - p4*k_1f*k_2f
            eq5 = k_1f * k_2f * k_3f * k_4f - p5
            eq6 = k_1r * k_2r * k_3r * k_4r - p6
            eq7 = k_1f + k_1r - p7
            eq8 = k_2f + k_2r - p8

            return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8]

        # Set the known values
        p1, p2, p3, p4, p5, p6, p7, p8 = [x for x in pVals.values()]+ [1, 1]

        # Call fsolve to find the values of the unknown variables
        x0 = (p1/p5, p2/p5, p3/p5, p4/p5, p6/p5, p7, p8, p7) # initial guess for variables
        bounds = tuple()
        for i in range(len(x0)):
            bounds = bounds+((0, np.inf),) # bounds for variables

        sol = least_squares(equations, x0, args=(p1, p2, p3, p4, p5, p6, p7, p8), bounds=(0, np.inf))

        kfkr_estimated={}
        keys=['k_1f', 'k_2f', 'k_3f', 'k_4f', 'k_1r', 'k_2r', 'k_3r', 'k_4r']
        kfkr_estimated['type 1'] = dict((el,sol.x[i]) for el,i in zip(keys,range(len(sol.x))))
        json_object = json.dumps(kfkr_estimated, indent=4, sort_keys=True,
                  separators=(', ', ': '), ensure_ascii=False, cls=NumpyEncoder)
        with open("./temporary files/kfkr_estimated.json", "w") as outfile:
            outfile.write(json_object)
            
    return

In [38]:
def singleKineticBuilder(bgTemplates):

    speciesNoDuplicateSingle = []
    for reaction in bgTemplates[singleSelection.value]['products']:
        for stoi,product in bgTemplates[singleSelection.value]['products'][reaction]:
            if product not in speciesNoDuplicateSingle:
                speciesNoDuplicateSingle.append(product)
    for reaction in bgTemplates[singleSelection.value]['reactants']:
        for stoi,reactant in bgTemplates[singleSelection.value]['reactants'][reaction]:
            if reactant not in speciesNoDuplicateSingle:
                speciesNoDuplicateSingle.append(reactant)
        
  
    N=np.zeros((len(speciesNoDuplicateSingle),len(bgTemplates[singleSelection.value]['products'])))
    
    for s in range(len(speciesNoDuplicateSingle)):
        for i,reaction in enumerate(bgTemplates[singleSelection.value]['products']):
            for stoi,product in bgTemplates[singleSelection.value]['products'][reaction]:
                if speciesNoDuplicateSingle[s]==product:
                    N[s][i] = float(stoi)
        for i,reaction in enumerate(bgTemplates[singleSelection.value]['reactants']):
            for stoi,reactant in bgTemplates[singleSelection.value]['reactants'][reaction]:
                if speciesNoDuplicateSingle[s]==reactant:
                    N[s][i] = -float(stoi)
    
    
    return [N,speciesNoDuplicateSingle]


In [39]:
def textFileGen(speciesNoDuplicateSingle,bgTemplates,kfkr_estimated,floatICget,N):

    # Create a text file (TEXT)

    unitsImport = ['def model composed_model as',
                   '\tdef import using "units_BG.cellml" for',
                    '\t\tunit concentrationUnit using unit concentrationUnit;',
                   '\t\tunit speciesConstantUnit using unit speciesConstantUnit;',
                   '\t\tunit fluxUnit using unit fluxUnit;',
                    '\tenddef;',
                  '\tdef comp main as']
    

    with open('./temporary files/yourCellMLmodel.txt', 'w') as fp:
        pass    
    with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
        for line in unitsImport:
            file.write(line+'\n')


    with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
        file.write('\t\tvar t: second {init: 0};'+'\n')


    for i,species in enumerate(speciesNoDuplicateSingle):
        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write('\t\tvar ' +species+': '+'concentrationUnit '+'{'+'init: '+str([x.value for x in floatICget if x.description==species][0])+'}'+';'+'\n')


    for reaction in bgTemplates[singleSelection.value]['reactants']:
        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write('\t\tvar '+reaction+': '+'fluxUnit '+';'+'\n')


    for k in kfkr_estimated[singleSelection.value]:
        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write('\t\tvar '+ k+': '+'fluxUnit '+'{'+'init: '+str(kfkr_estimated[singleSelection.value][k])+'}'+';'+'\n')


    # Generating the ODEs       
    with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
        file.write('\n')

    for i,species in enumerate(speciesNoDuplicateSingle):
        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write('\t\tode('+species+',t)=')
        for j in range(np.shape(N)[1]):  
            if N[i][j] != 0:

                if N[i][j] > 0:
                    with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
                        file.write('+'+str(N[i][j])+'{dimensionless}*'+list(bgTemplates[singleSelection.value]['products'].keys())[j])

                if N[i][j] < 0:
                    with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
                        file.write(str(N[i][j])+'{dimensionless}*'+list(bgTemplates[singleSelection.value]['products'].keys())[j])

        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write(';\n')


    # Generating the reaction rates
    with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
        file.write('\n')

    for r,flux in enumerate(bgTemplates[singleSelection.value]['products'].keys()):
        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write('\t\t '+ flux+'= ') 
                    
        
#         if singleSelection.value=='SGLT1' and 'sodium(1+)' in meaning[flux+'_'+singleSelection.value]:
#             z_Na=1; F=96485; T=310; R=8.314; V_na=-0.046;
#             membraneContrib = math.exp((z_Na*F*V_na)/(R*T))
#             with open('yourCellMLmodel.txt', 'a') as file:
#                 file.write(str(membraneContrib)+'{dimensionless}'+'*')

        
        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write('k_'+str(r+1)+'f'+'*(')
        length = 0    
        for stoi,reactant in bgTemplates[singleSelection.value]['reactants'][flux]:
            length+=1
            with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
                file.write('pow('+reactant+','+str(stoi)+'{dimensionless}'+')') 

            if length < len(bgTemplates[singleSelection.value]['reactants'][flux]):
                with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
                    file.write(' * ')



        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write(') - ')
            
            
        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write('k_'+str(r+1)+'r'+'*(') 
        length = 0
        for stoi,product in bgTemplates[singleSelection.value]['products'][flux]:
            length+=1
            with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
                file.write('pow('+product+','+str(stoi)+'{dimensionless}'+')') 

            if length < len(bgTemplates[singleSelection.value]['products'][flux]):
                with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
                    file.write(' * ')

        with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
            file.write(');\n')


    with open('./temporary files/yourCellMLmodel.txt', 'a') as file:
        file.write('\tenddef;'+'\n'+'enddef;')
        
    return 

In [40]:
# Generating a json file for BG templates
F=96485
type2_z_B=1

types = ['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9']
reactions = {'type 1':['Re1', 'Re2', 'Re3', 'Re4'], 
            'type 2':['Re1', 'Re2', 'Re3', 'Re4', 'Re5', 'Re6'], 
            'type 3':['Re1', 'Re2', 'Re3', 'Re4', 'Re5', 'Re6'], 
            'type 4':['Re1', 'Re2', 'Re3', 'Re4', 'Re5', 'Re6', 'Re7', 'Re8']}

X = dict((el,dict((el1,{}) for el1 in ['reactants','products'])) for el in types)


X['type 1']['reactants']={'Re1': [(1,'EAi')], 'Re2': [(1,'Eo'),(1,'Ao')], 'Re3': [(1,'Ei')], 'Re4': [(1,'EAo')]}
X['type 1']['products']={'Re1': [(1,'Ai'),(1,'Ei')], 'Re2': [(1,'EAo')], 'Re3': [(1,'Eo')], 'Re4': [(1,'EAi')]}

X['type 2']['reactants']={'Re1': [(1,'Eo'),(1,'Ao')], 'Re2': [(1,'EAi')], 'Re3': [(1,'EAo'),(1,'Bo'),(type2_z_B*F,'Vm')], 
                          'Re4': [(1,'EBAi')], 'Re5': [(1,'EBAo')], 'Re6': [(1,'Ei')]}
X['type 2']['products']={'Re1': [(1,'EAo')], 'Re2': [(1,'Ai'),(1,'Ei')], 'Re3': [(1,'EBAo')], 
                         'Re4': [(1,'EAi'),(1,'Bi')], 'Re5': [(1,'EBAi')], 'Re6': [(1,'Eo')]}

X['type 3']['reactants']={'Re1': [(1,'Eo'),(1,'Ao')], 'Re2': [(1,'EAi')], 'Re3': [(1,'EBo')], 
                          'Re4': [(1,'Bi'),(1,'Ei')], 'Re5': [(1,'EBi')], 'Re6': [(1,'EAo')]}
X['type 3']['products']={'Re1': [(1,'EAo')], 'Re2': [(1,'Ai'),(1,'Ei')], 'Re3': [(1,'Bo'),(1,'Eo')], 
                         'Re4': [(1,'EBi')], 'Re5': [(1,'EBo')], 'Re6': [(1,'EAi')]}

X['type 4']['reactants']={'Re1': [(1,'EAo')], 'Re2': [(1,'Ai'),(1,'Ei')], 'Re3': [(1,'Eo'),(1,'Bo')], 'Re4': [(1,'EBi')], 
                          'Re5': [(1,'EBo'),(2,'Co')], 'Re6': [(1,'EBCi')], 'Re7': [(1,'EAi')], 'Re8': [(1,'EBCo')]}
X['type 4']['products']={'Re1': [(1,'Ao'),(1,'Eo')], 'Re2': [(1,'EAi')], 'Re3': [(1,'EBo')], 'Re4': [(1,'Ei'),(1,'Bi')], 
                         'Re5': [(1,'EBCo')], 'Re6': [(1,'EBi'),(2,'Ci')], 'Re7': [(1,'EAo')], 'Re8': [(1,'EBCi')]}
    

json_object = json.dumps(X, indent=4, sort_keys=True,
      separators=(', ', ': '), ensure_ascii=False,cls=NumpyEncoder)
with open("./jsonTemplates/bgTemplates.json", "w") as outfile:
    outfile.write(json_object)

# Section 1: Model selection


In [41]:
# Model Selection and BG structure demonstration

singleSelection = widgets.Dropdown(
            options=['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9'],
            description='Select BG type:',
            style={'description_width': 'initial'},
            disabled=False
        )
button_load = widgets.Button(
                description='Load template',
                tooltip='Load',
                style={'description_width': 'initial'},
                button_style='info',
                layout = Layout(width='200px', height='40px')
            )
button_load.style.font_family = 'monospace'
button_load.style.font_size = '20px'
output1 = widgets.Output(layout={'border': '1px solid gray'})

def button_load_clicked(event):
    with output1:
        clear_output()

        # Define the path of the folder containing the images you want to display
        folder_path = './figures'

        for file in os.listdir(folder_path):
            if file.strip('.jpg') == singleSelection.value or \
            file.strip('.JPG') == singleSelection.value or \
            file.strip('.JPEG') == singleSelection.value or \
            file.strip('.jpeg') == singleSelection.value: 
                file_path = os.path.join(folder_path, file)
                img = Image.open(file_path)
                new_image = img.resize((800, 500))
                text_0 = widgets.HTML(value="<h5><b>Bond graph structure of {}:<b><h5>".format(file.strip('.jpg')))
                display(text_0,new_image)

        
button_load.on_click(button_load_clicked)
vbox_result = widgets.VBox([button_load, output1])   

vbox_text = widgets.VBox([singleSelection, vbox_result])
page1 = widgets.HBox([vbox_text])

display(page1)

# Section 2: Composite Annotation

In [42]:
types = ['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9']

valueRequired1 = dict((el,[]) for el in types)
valueRequired1['type 1']=['Ao','Ai']
valueRequired1['type 2']=['Ao','Ai','Bo','Bi','z_B','Vm']
valueRequired1['type 3']=['Ao','Ai','Bo','Bi']
valueRequired1['type 4']=['Ao','Ai','Bo','Bi','Co','Ci']


button_annotate = widgets.Button(
                description='Start annotating!',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='info',
                layout = Layout(width='250px', height='40px')
            )
button_annotate.style.font_family = 'monospace'
button_annotate.style.font_size = '20px'
output_annotations= widgets.Output(layout={'height': '100%'})
    


def button_annotate_clicked(event):
    with output_annotations:
        # output_annotations.clear_output()
        text00=widgets.HTML(value="<h3><b><font color='maroon'>Annotate the following components as in the loaded template:<b><h3>")
        display(text00)
        selectedAnnotations = {}
        i=-1
        if singleSelection.value not in selectedAnnotations.keys():
            key = singleSelection.value
            selectedAnnotations[key]={}
        else:
            i+=1
            key = singleSelection.value+'_'+str(i)
            selectedAnnotations[key]={}
        for element in valueRequired1[singleSelection.value]:
            selectedAnnotations[key][element]=[]
            text=widgets.HTML(value="<h3><b><font color='teal'>{}<b><h3>".format(element))
            display(text)
            selectedAnnotations = annotate(key,element,selectedAnnotations)
            
            # Save the values in .json file
        with open("./temporary files/selectedAnnotations.json", "w") as outfile:
            json.dump(selectedAnnotations, outfile, indent=4, sort_keys=True, ensure_ascii=False)

            
def annotate(key,element,selectedAnnotations):
    # Create a text input widget for searching
    search_widget_left = widgets.Text(
        placeholder='Enter search term...',
        description='Keyword:'
    )
    search_widget_middle = widgets.Text(
        placeholder='Enter search term...',
        description='Keyword:'
    )
    search_widget_right = widgets.Text(
        placeholder='Enter search term...',
        description='Keyword:'
    )
    
    # Create radio buttons for selecting from retrieved values
    radio_buttons_left = widgets.RadioButtons(
        options=[],
        description='Select from retrieved values:',
        disabled=False,
        style={'description_width': 'initial'}
    )
    radio_buttons_middle = widgets.RadioButtons(
        options=[],
        description='Select from retrieved values:',
        disabled=False,
        style={'description_width': 'initial'}
    )
    radio_buttons_right = widgets.RadioButtons(
        options=[],
        description='Select from retrieved values:',
        disabled=False 
    )


    # Create a dropdown widget for selecting the search column
    column_names_left = ['Preferred Label','Class ID']
    column_names_widget_left = widgets.Dropdown(
        options=['Preferred Label','Class ID'],
        value='Preferred Label',
        description='Search column:',
        style={'description_width': 'initial'}
    )
    column_names_middle = ['Preferred Label','Class ID']
    column_names_widget_middle = widgets.Dropdown(
        options=['Preferred Label','Class ID'],
        value='Preferred Label',
        description='Search column:',
        style={'description_width': 'initial'}
    )
    column_names_right = ['Preferred Label','Class ID']
    column_names_widget_right = widgets.Dropdown(
        options=['Preferred Label','Class ID'],
        value='Preferred Label',
        description='Search column:',
        style={'description_width': 'initial'}
    )

    # Create an output widget for displaying search results
    output_widget_left = widgets.Output(layout={'height': '150px', 'overflow': 'scroll'})
    output_widget_middle= widgets.Output(layout={'height': '150px', 'overflow': 'scroll'})
    output_widget_right = widgets.Output(layout={'height': '150px', 'overflow': 'scroll'})

    # Define a function to handle search events
    def handle_search_left(sender1):
        with output_widget_left:
            output_widget_left.clear_output()
            df_left = pd.read_csv('OPB.csv')
            search_term_left = search_widget_left.value.lower()
            column_name_left = column_names_widget_left.value
            filtered_df_left = df_left[df_left[column_name_left].str.lower().str.contains(search_term_left)]
            radio_buttons_left.options = [z+':  '+y for z,y in zip([x.replace('http://bhi.washington.edu/OPB#',"") for x in filtered_df_left['Class ID'].tolist()],filtered_df_left['Preferred Label'].tolist())]
            radio_buttons_left.disabled = False
            radio_buttons_left.layout = Layout(width='100%', height='100%')
            display(radio_buttons_left)

    def handle_search_middle(sender2):
        with output_widget_middle:
            output_widget_middle.clear_output()
            df_middle = pd.read_csv('CHEBI.csv')
            search_term_middle = search_widget_middle.value.lower()
            column_name_middle = column_names_widget_middle.value
            filtered_df_middle = df_middle[df_middle[column_name_middle].astype(str).str.lower().str.contains(search_term_middle.lower(), na=False)]
            radio_buttons_middle.options = [z+':  '+y for z,y in zip([x.replace('http://purl.obolibrary.org/obo/',"") for x in filtered_df_middle['Class ID'].tolist()],filtered_df_middle['Preferred Label'].tolist())]
            radio_buttons_middle.disabled = False
            radio_buttons_middle.layout = Layout(width='100%', height='100%')
            display(radio_buttons_middle)

    def handle_search_right(sender3):
        with output_widget_right:
            output_widget_right.clear_output()
            df_right = pd.read_csv('FMA.csv')
            search_term_right = search_widget_right.value.lower()
            column_name_right = column_names_widget_right.value
            filtered_df_right = df_right[df_right[column_name_right].astype(str).str.lower().str.contains(search_term_right.lower(), na=False)]
            radio_buttons_right.options = [z+':  '+y for z,y in zip([x.replace('http://purl.org/sig/ont/fma/',"") for x in filtered_df_right['Class ID'].tolist()],filtered_df_right['Preferred Label'].tolist())]
            radio_buttons_right.disabled = False
            radio_buttons_right.layout = Layout(width='100%', height='100%')
            display(radio_buttons_right)
            
            
            
            
    def handle_selection_left(change1):
        for i in selectedAnnotations[key][element]:
            if 'OPB' in i:
                selectedAnnotations[key][element].remove(i)
        selectedAnnotations[key][element].append(change1.new)
        with open('./temporary files/selectedAnnotations.json', 'w') as f1:
            json.dump(selectedAnnotations, f1, indent=4, sort_keys=True, ensure_ascii=False)      
    def handle_selection_middle(change2):
        for i in selectedAnnotations[key][element]:
            if 'CHEBI' in i:
                selectedAnnotations[key][element].remove(i)
        selectedAnnotations[key][element].append(change2.new)
        with open('./temporary files/selectedAnnotations.json', 'w') as f2:
            json.dump(selectedAnnotations, f2, indent=4, sort_keys=True, ensure_ascii=False)  
    def handle_selection_right(change3):
        for i in selectedAnnotations[key][element]:
            if 'fma' in i:
                selectedAnnotations[key][element].remove(i)
        selectedAnnotations[key][element].append(change3.new)
        with open('./temporary files/selectedAnnotations.json', 'w') as f3:
            json.dump(selectedAnnotations, f3, indent=4, sort_keys=True, ensure_ascii=False)  


    # Attach the search function to the text input widget's submit event
    search_widget_left.on_submit(handle_search_left)
    search_widget_middle.on_submit(handle_search_middle)
    search_widget_right.on_submit(handle_search_right)
    
    radio_buttons_left.observe(handle_selection_left, names='value')
    radio_buttons_middle.observe(handle_selection_middle, names='value')
    radio_buttons_right.observe(handle_selection_right, names='value')

    # Display the widgets
    text_left=widgets.HTML(value="<h4><b><font color='salmon'>Physical property:<b><h4>")
    display(text_left)
    display(search_widget_left, column_names_widget_left)
    display(output_widget_left)
    text_middle=widgets.HTML(value="<h4><b><font color='salmon'>Physical entity:<b><h4>")
    display(text_middle)
    display(search_widget_middle, column_names_widget_middle)
    display(output_widget_middle)
    text_right=widgets.HTML(value="<h4><b><font color='salmon'>Contained in:<b><h4>")
    display(text_right)
    display(search_widget_right, column_names_widget_right)
    display(output_widget_right)
    
    
    return selectedAnnotations
    
            
button_annotate.on_click(button_annotate_clicked)
annoat_page = widgets.VBox([button_annotate,output_annotations])
display(annoat_page)

# Section 3: The Initial Conditions


In [11]:
# Getting the initial conditions
types = ['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9']

valueRequired1 = dict((el,[]) for el in types)
valueRequired1['type 1']=['Ao','Ai']
valueRequired1['type 2']=['Ao','Ai','Bo','Bi','z_B','Vm']
valueRequired1['type 3']=['Ao','Ai','Bo','Bi']
valueRequired1['type 4']=['Ao','Ai','Bo','Bi','Co','Ci']

Enzymes = dict((el,[]) for el in types)
Enzymes['type 1']=['Ei','Eo','EAi','EAo']
Enzymes['type 2']=['Ei','Eo','EAi','EAo','EBAo','EBAi']
Enzymes['type 3']=['Ei','Eo','EAi','EAo','EBo','EBi']
Enzymes['type 4']=['Ei','Eo','EAi','EAo','EBo','EBi','EBCo','EBCi']


E_tot = widgets.FloatLogSlider(
    value=1e-6,
    base=10,
    min=-6, # max exponent of base
    max=0, # min exponent of base
    step=0.05, # exponent step
    description='Total Enzyme Concentration',
    layout={'width': 'auto'},
    style={'description_width': 'initial'},
)
E_tot.style.handle_color = 'lightblue'
display(E_tot)
button_saveEtot = widgets.Button(
                description='Save',
                tooltip='Save',
                style={'description_width': 'initial'},
                layout = Layout(width='120px', height='40px'))
button_saveEtot.style.font_family = 'monospace'
button_saveEtot.style.font_size = '20px'
button_saveEtot.style.button_color = 'lightgreen'
output7 = widgets.Output(layout={'border': '1px solid gray'})



button_saveIC = widgets.Button(
                description='Confirm',
                tooltip='Confirm',
                style={'description_width': 'initial'},
            layout = Layout(width='120px', height='40px'))
button_saveIC.style.font_family = 'monospace'
button_saveIC.style.font_size = '20px'
button_saveIC.style.button_color = 'lightgreen'

def button_saveIC_clicked(button_saveIC):
    with output7:
        clear_output()
        print('The total enzyme concentration has been saved.')
        print('Insert the initial concentrations.')

        global E_tot, floatICget
        floatICget=[]
        for i,el in enumerate(valueRequired1[singleSelection.value]):
            floatICget.append(widgets.FloatText(
                    value=0.0,
                    description=el,
                    disabled=False
                ))
        for i,el in enumerate(Enzymes[singleSelection.value]):
            floatICget.append(widgets.FloatText(
                    value=E_tot.value/len(Enzymes[singleSelection.value]),
                    description=el,
                    disabled=False
                ))
            
        output8 = widgets.Output(layout={})
        
        display(*floatICget)
        def button_saveEtot_clicked(event):
            with output8:
                clear_output()
                text = widgets.HTML(value="<h4><b>The initial conditions have been saved.<b><h4>")
                display(text)  
                IC = dict((x.description,x.value) for x in floatICget)
                initialValues = {singleSelection.value: IC}
                with open("./temporary files/initialValues.json", "w") as outfile:
                    json.dump(initialValues, outfile, indent=4, sort_keys=True, ensure_ascii=False)
        button_saveEtot.on_click(button_saveEtot_clicked)

        display(button_saveEtot)
        display(output8)
        

button_saveIC.on_click(button_saveIC_clicked)
    
    
vbox_result = widgets.VBox([button_saveIC, output7])   

pagei = widgets.HBox([vbox_result])

display(pagei)
   




FloatLogSlider(value=1e-06, description='Total Enzyme Concentration', layout=Layout(width='auto'), max=0.0, mi…

# Section 4: The Steady-State Values



In [12]:
# Getting the SS values
types = ['type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8', 'type 9']
valueRequired = dict((el,[]) for el in types)

valueRequired['type 1']=['V_SS','Ao','Ai']
valueRequired['type 2']=['V_SS','Ao','Ai','Bo','Bi']
valueRequired['type 3']=['V_SS','Ao','Ai','Bo','Bi']
valueRequired['type 4']=['V_SS','Ao','Ai','Bo','Bi','Co','Ci']

floatSSget=[]
for i,el in enumerate(valueRequired[singleSelection.value]):
    floatSSget.append(widgets.FloatText(
            value=0.0,
            description=el,
            disabled=False
        ))
button_saveSS = widgets.Button(
                description='Save',
                tooltip='Save',
                style={'description_width': 'initial'},
                layout = Layout(width='120px', height='40px'))
button_saveSS.style.font_family = 'monospace'
button_saveSS.style.font_size = '20px'
button_saveSS.style.button_color = 'lightgreen'
output2 = widgets.Output(layout={})

def button_saveSS_clicked(event):
    with output2:
        clear_output()
        text = widgets.HTML(value="<h5><b>The inserted steady-state values have been saved.<b><h5>")
        display(text)      
        pVals(singleSelection,floatSSget)
        kfkr_estimation(singleSelection)

button_saveSS.on_click(button_saveSS_clicked)
display(*floatSSget)

vbox_result = widgets.VBox([button_saveSS, output2])   
page6 = widgets.HBox([vbox_result])
display(page6)




FloatText(value=0.0, description='V_SS')

FloatText(value=0.0, description='Ao')

FloatText(value=0.0, description='Ai')

# Section 5: Stimulus

In [12]:
# Define the ODEs

button_stim = widgets.Button(
                description='Run',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='warning',
                layout = Layout(width='120px', height='40px'))
button_stim.style.font_family = 'monospace'
button_stim.style.font_size = '20px'
outputStim = widgets.Output(layout={'border': '1px solid gray'})



def solveODE(event):
    with outputStim:
        outputStim.clear_output()
        img = Image.open('./figures/stimuli/step.jpg')
        display(img.resize((200, 200)))
        def type1Control(t, variables, type1Data, step_func):    
            k_1f, k_1r, k_2f, k_2r, k_3f, k_3r, k_4f, k_4r = type1Data
            Ai, Ei, EAo, Eo, EAi = variables

            Ao = step_func(t) # Add step function for Ao

            Re1 = k_1f*(pow(EAi,1)) - k_1r*(pow(Ai,1) * pow(Ei,1)); 
            Re2 = k_2f*(pow(Eo,1) * pow(Ao,1)) - k_2r*(pow(EAo,1));
            Re3 = k_3f*(pow(Ei,1)) - k_3r*(pow(Eo,1));
            Re4 = k_4f*(pow(EAo,1)) - k_4r*(pow(EAi,1));

            dAidt = Re1
            dEidt = Re1 - Re3
            dEAodt = Re2 - Re4
            dEodt = -Re2 + Re3
            dEAidt = -Re1 + Re4
            # dAodt = -Re2

            return [dAidt, dEidt, dEAodt, dEodt, dEAidt]

        # Set the known values
        f = open("./temporary files/kfkr_estimated.json")
        kfkr_estimated = json.load(f)
        type1Data = kfkr_estimated['type 1'].values()

        # Set the initial conditions
        f = open("./temporary files/initialValues.json")
        initialValues = json.load(f)
        type1x0=[]
        for el in ['Ai', 'Ei', 'EAo', 'Eo', 'EAi']:
            for species in initialValues['type 1'].keys():
                if el==species:
                    type1x0.append(initialValues['type 1'][species])

        
        runTime = FloatSlider(value=1000,min=0,max=1000,step=0.01,layout=Layout(width='100%'), description='Run time:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='.1f')
        stepTime=FloatSlider(min=0,max=100,step=0.001,layout=Layout(width='100%'), readout_format='.3f', style={'description_width': 'initial'}, description='t0')        
        stepAmplitude=FloatSlider(min=0,max=10000,step=10,layout=Layout(width='100%'), readout_format='.3f', style={'description_width': 'initial'}, description='y0')
        def plotf(runTime,stepTime,stepAmplitude):
            # Define the step function for Ao
            def step_func(t):
                if t < stepTime:
                    return [x.value for x in floatICget if x.description=='Ao'][0]
                else:
                    return stepAmplitude

            # Set the time points where you want to evaluate the solution
            t_eval = np.arange(0,runTime,0.1)

            # Solve the ODEs
            solution = solve_ivp(type1Control, (0,runTime), type1x0, method=LSODA, args=(type1Data,step_func),  t_eval=t_eval)
            plt.figure(figsize=(5,5))
            plt.plot(t_eval, solution.y[0,:],label='Ai',color='r', linewidth=2) 
            plt.grid(color='gray', linestyle='-', linewidth=0.5)
            plt.ylabel('Concentration')
            plt.legend(loc='best')
            plt.show()
            return 

        interact(plotf, runTime=runTime, stepTime=stepTime, stepAmplitude=stepAmplitude)
        
        
button_stim.on_click(solveODE)
vbox_result = widgets.VBox([button_stim, outputStim]) 
page7 = widgets.HBox([vbox_result])
display(page7)

# Section 6: CellML Code for Single Models


In [155]:
button_cellml = widgets.Button(
                description='Generate code',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='danger',
                layout = Layout(width='180px', height='40px'))
button_cellml.style.font_family = 'monospace'
button_cellml.style.font_size = '20px'
output6 = widgets.Output(layout={'border': '1px solid gray'})

def on_button_clicked6(event):
    with output6:
        clear_output()
        f = open("./temporary files/kfkr_estimated.json")
        kfkr_estimated = json.load(f)
        f = open("./jsonTemplates/bgTemplates.json")
        bgTemplates = json.load(f)
        
        [N,speciesNoDuplicateSingle] = singleKineticBuilder(bgTemplates)
        tempDict={}
        tempDict[singleSelection.value] = N.tolist()
        with open("./temporary files/N.json", "w") as outfile:
            json.dump(tempDict, outfile, indent=4, sort_keys=True, ensure_ascii=False)
        speciesNoDuplicateSingleDict={}
        speciesNoDuplicateSingleDict[singleSelection.value]=speciesNoDuplicateSingle
        with open('./temporary files/speciesNoDuplicateSingleDict.json', 'w') as f:
            json.dump(speciesNoDuplicateSingleDict, f, indent=4, sort_keys=True, ensure_ascii=False)      
        
        textFileGen(speciesNoDuplicateSingle,bgTemplates,kfkr_estimated,floatICget,N)
        text_0 = widgets.HTML(value="<h3>The CellML code for the bond graph form of {} is:</h3>".format(singleSelection.value))
        vbox_text = widgets.VBox([text_0])
        display(vbox_text)
        with open('./temporary files/yourCellMLmodel.txt') as f:
            contents = f.read()
        print(contents)

button_cellml.on_click(on_button_clicked6)
vbox_result = widgets.VBox([button_cellml, output6]) 
display(vbox_result)


# Section 7: Save the Model
#### Add the generated model to a list for model composition

In [15]:
button_modelName = widgets.Textarea(
                description='Give it a name:',
                tooltip='Description',
                style={'description_width': 'initial'})
display(button_modelName)


button_addModel = widgets.Button(
                description='Save the model',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout = Layout(width='180px', height='40px'))
button_addModel.style.font_family = 'monospace'
button_addModel.style.font_size = '20px'
button_addModel.style.button_color = 'lightgreen'
output9 = widgets.Output(layout={})

def button_addModel_clicked(event):
    with output9:
        output9.clear_output()
        if button_modelName.value.strip('\n') not in savedModels and button_modelName.value.strip('\n')!='':
            savedModels.append(button_modelName.value.strip('\n'))
            text = widgets.HTML(value="<h3>Your generated model has been saved.</h3>")
            display(text)
            # Save the annotations as a new json file
            f = open("./temporary files/selectedAnnotations.json")
            file = json.load(f)
            file[button_modelName.value.strip('\n')]=file[singleSelection.value]
            del file[singleSelection.value]
            savedAnnotations.update(file)
            with open('./savedModels/savedAnnotations.json', 'w') as f:
                json.dump(savedAnnotations, f, indent=4, sort_keys=True, ensure_ascii=False)  
            
            # Save the initial values as a new json file
            f = open("./temporary files/initialValues.json")
            file = json.load(f)
            file[button_modelName.value.strip('\n')]=file[singleSelection.value]
            del file[singleSelection.value]
            initialValues.update(file)
            with open('./savedModels/savedInitialValues.json', 'w') as f:
                json.dump(initialValues, f, indent=4, sort_keys=True, ensure_ascii=False)  
                
            # Save the N matrices as a new json file
            f = open("./temporary files/N.json")
            file = json.load(f)
            file[button_modelName.value.strip('\n')]=file[singleSelection.value]
            del file[singleSelection.value]
            Nmatrices.update(file)
            with open('./savedModels/savedNmatrices.json', 'w') as f:
                json.dump(Nmatrices, f, indent=4, sort_keys=True, ensure_ascii=False)  
    
    
            # Save the affinities as a new json file
            f = open("./temporary files/kfkr_estimated.json")
            file = json.load(f)
            file[button_modelName.value.strip('\n')]=file[singleSelection.value]
            del file[singleSelection.value]
            savedkfkr.update(file)
            with open('./savedModels/savedkfkr.json', 'w') as f:
                json.dump(savedkfkr, f, indent=4, sort_keys=True, ensure_ascii=False)  
            # Save the speciesNoDuplicateSingle as a new json file
            f = open("./temporary files/speciesNoDuplicateSingleDict.json")
            file = json.load(f)
            file[button_modelName.value.strip('\n')]=file[singleSelection.value]
            del file[singleSelection.value]
            savedSpeciesNoDuplicateSingleDict.update(file)
            with open('./savedModels/savedSpeciesNoDuplicateSingleDict.json', 'w') as f:
                json.dump(savedSpeciesNoDuplicateSingleDict, f, indent=4, sort_keys=True, ensure_ascii=False)  
            # Save the type as a new json file    
            file=dict([(button_modelName.value.strip('\n'),singleSelection.value)])
            savedtypes.update(file)
            with open('./savedModels/savedtypes.json', 'w') as f:
                json.dump(savedtypes, f, indent=4, sort_keys=True, ensure_ascii=False)
            # Save the single cellml model 
            with open('./temporary files/yourCellMLmodel.txt') as f:
                contents = f.read()
            with open('./savedModels/{}.txt'.format(button_modelName.value.strip('\n')), 'w') as file:
                file.write(contents)
            
        elif button_modelName.value.strip('\n') == '':    
            text = widgets.HTML(value="<h3><font color='red'>None: Enter a valid name.</h3>")
            display(text)
        else:
            text = widgets.HTML(value="<h3><font color='red'>Repetitive name: Enter a new name.</h3>")
            display(text)
        
        print('Your saved models are:')
        print(savedModels)
        
button_addModel.on_click(button_addModel_clicked)
vbox_result = widgets.VBox([button_addModel, output9]) 
display(vbox_result)

Textarea(value='', description='Give it a name:', style=TextStyle(description_width='initial'), tooltip='Descr…

# Section 8: Model Composition
#### Compose the saved models
###### Composition occurs if the models have mutual entities

In [13]:
button_compose = widgets.Button(
                description='Compose the models',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='danger',
                layout = Layout(width='220px', height='40px'))
button_compose.style.font_family = 'monospace'
button_compose.style.font_size = '20px'
output10 = widgets.Output(layout={})

def button_compose_clicked(event):
    with output10:
        output10.clear_output()
        f = open("./jsonTemplates/bgTemplates.json")
        bgTemplates = json.load(f)
        f = open("./savedModels/savedAnnotations.json")
        savedAnnotations = json.load(f)
        f = open("./savedModels/savedtypes.json")
        savedtypes = json.load(f)
        f = open("./savedModels/savedkfkr.json")
        savedkfkr = json.load(f)    
        f = open("./savedModels/savedSpeciesNoDuplicateSingleDict.json")
        savedSpeciesNoDuplicateSingleDict = json.load(f)
        f = open("./savedModels/savedNmatrices.json")
        savedNmatrices = json.load(f)
        f = open("./savedModels/savedInitialValues.json")
        savedInitialValues = json.load(f)


        knowns=[]
        for modelName,Type in savedtypes.items():
            for r,reaction in enumerate(bgTemplates[Type]['reactants'].keys()):
                knowns.append(savedkfkr[modelName]['k_'+str(r+1)+'f']/savedkfkr[modelName]['k_'+str(r+1)+'r'])

        unknowns=[]
        # adding the unannotated species to the unknowns list for K_(species)
        for modelName,Type in savedtypes.items():
            for speciesID in savedSpeciesNoDuplicateSingleDict[modelName]:
                if speciesID not in savedAnnotations[modelName].keys():
                    unknowns.append(modelName+':'+speciesID)



        toRemove = {}
        same_rdf={}
        for key1, value1 in savedAnnotations.items():
            for key11,value11 in value1.items():
                for key2, value2 in savedAnnotations.items():
                    for key22,value22 in value2.items():
                        if key22 not in same_rdf.keys():
                            same_rdf[key22]=[]
                        else:
                            if key1 != key2 and value11 == value22:
                                if key1 not in same_rdf[key22]:
                                    same_rdf[key22].append(key1)
                                if key2 not in same_rdf[key22]:
                                    same_rdf[key22].append(key2)

        for key,value in same_rdf.items():
            toRemove[key] = value[:-1]

        for modelName1,value in savedAnnotations.items():
            for ID1 in value.keys():
                if ID1 in toRemove.keys() and  modelName1 not in toRemove[ID1]:
                    unknowns.append(modelName1+':'+ID1)

                if ID1 not in toRemove.keys():
                    unknowns.append(modelName1+':'+ID1)


        M = np.zeros((len(knowns),len(unknowns)))
        length=0
        for modelName,Type in savedtypes.items():
            for r,reaction in enumerate(bgTemplates[Type]['reactants']):
                for stoi,species in bgTemplates[Type]['reactants'][reaction]:
                    for k1,K_species in enumerate(unknowns):
                        if modelName+':'+species == K_species:
                            M[length+r][k1] = stoi
                        if species in toRemove.keys() and modelName in toRemove[species]:
                            if modelName in same_rdf[species]:
                                for model in same_rdf[species]:
                                    if model != modelName and model+':'+species == K_species:
                                        M[length+r][k1] = stoi

                for stoi,species in bgTemplates[Type]['products'][reaction]:
                    for k2,K_species in enumerate(unknowns):
                        if modelName+':'+species == K_species: 
                            M[length+r][k2] = -stoi
                        if species in toRemove.keys() and modelName in toRemove[species]:
                            if modelName in same_rdf[species]:
                                for model in same_rdf[species]:
                                    if model != modelName and model+':'+species == K_species:
                                        M[length+r][k2] = -stoi

            length += r+1


        logTrueValues = [];
        for k in knowns:
            logTrueValues.append(math.log(k))
        logTrueValues = np.array(logTrueValues)

        Mpinv = np.linalg.pinv(np.array(M))
        logU = Mpinv.dot(logTrueValues)

        thermodynamics = [math.exp(i) for i in list(logU)]   
            
        # Finding the reaction rate constants 

        def type1ODEs(t,variables, type1Data):    
            k_1f, k_1r, k_2f, k_2r, k_3f, k_3r, k_4f, k_4r = type1Data
            Ai, Ao, Ei, EAo, Eo, EAi = variables


            Re1 = k_1f*(pow(EAi,1)) - k_1r*(pow(Ai,1) * pow(Ei,1)); 
            Re2 = k_2f*(pow(Eo,1) * pow(Ao,1)) - k_2r*(pow(EAo,1));
            Re3 = k_3f*(pow(Ei,1)) - k_3r*(pow(Eo,1));
            Re4 = k_4f*(pow(EAo,1)) - k_4r*(pow(EAi,1));

            dAidt = Re1
            dEidt = Re1 - Re3
            dEAodt = Re2 - Re4
            dEodt = -Re2 + Re3
            dEAidt = -Re1 + Re4
            dAodt = -Re2

            return [dAidt, dAodt, dEidt, dEAodt, dEodt, dEAidt]


        Solutions={}
        for modelName,Type in savedtypes.items():
            Solutions[modelName]={}
            if Type == 'type 1':
                type1x0=[]
                for el in ['Ai', 'Ao', 'Ei', 'EAo', 'Eo', 'EAi']:
                    for species in savedInitialValues[modelName].keys():
                        if el==species:
                            type1x0.append(savedInitialValues[modelName][species])

                type1Data = tuple(savedkfkr[modelName].values())
                t_eval = np.arange(0,100,0.1)

                # Solve the ODEs
                solution = solve_ivp(lambda t, y: type1ODEs(t, y, type1Data), (0, 100), type1x0, method='LSODA', t_eval=t_eval, dense_output=True)

                Solutions[modelName]['Re1'] = savedkfkr[modelName]['k_1f']*(pow(solution.y[5],1)) - savedkfkr[modelName]['k_1r']*(pow(solution.y[0],1) * pow(solution.y[2],1)); 
                Solutions[modelName]['Re2'] = savedkfkr[modelName]['k_2f']*(pow(solution.y[4],1) * pow(solution.y[1],1)) - savedkfkr[modelName]['k_2r']*(pow(solution.y[3],1));
                Solutions[modelName]['Re3'] = savedkfkr[modelName]['k_3f']*(pow(solution.y[2],1)) - savedkfkr[modelName]['k_3r']*(pow(solution.y[4],1));
                Solutions[modelName]['Re4'] = savedkfkr[modelName]['k_4f']*(pow(solution.y[3],1)) - savedkfkr[modelName]['k_4r']*(pow(solution.y[5],1));

                Solutions[modelName]['Ai'] = solution.y[0]
                Solutions[modelName]['Ao'] = solution.y[1]
                Solutions[modelName]['Ei'] = solution.y[2]
                Solutions[modelName]['EAo'] = solution.y[3]
                Solutions[modelName]['Eo'] = solution.y[4]
                Solutions[modelName]['EAi'] = solution.y[5]



        reactionRates = {}

        for modelName,Type in savedtypes.items():
            reactionRates[modelName]={}
            for r,reaction in enumerate(bgTemplates[Type]['reactants']): # Reactions
                reactantsSpecies=[]
                productsSpecies=[]

                K_reactants = 1
                K_products = 1

                solve=tuple()

                for stoichiometry,reactant in bgTemplates[Type]['reactants'][reaction]:
                    reactantsSpecies.append(reactant)      
                    solve=solve+(pow(Solutions[modelName][reactant],stoichiometry),)

                    for i in range(len(unknowns)):
                        if unknowns[i] == modelName+':'+reactant:
                            K_reactants = K_reactants * pow(thermodynamics[i],stoichiometry)  
                            break
                        if reactant in toRemove.keys() and modelName in toRemove[reactant]:
                            if modelName in same_rdf[reactant]:
                                for model in same_rdf[reactant]:
                                    if model != modelName and model+':'+reactant == unknowns[i]:
                                        K_reactants = K_reactants * pow(thermodynamics[i],stoichiometry)
                                        break

                for stoichiometry,product in bgTemplates[Type]['products'][reaction]:
                    productsSpecies.append(product)      
                    solve=solve+(pow(Solutions[modelName][product],stoichiometry),)

                    for i in range(len(unknowns)):
                        if unknowns[i] == modelName+':'+product:
                            K_products = K_products * pow(thermodynamics[i],stoichiometry)  
                            break
                        if product in toRemove.keys() and modelName in toRemove[product]:
                            if modelName in same_rdf[product]:
                                for model in same_rdf[product]:
                                    if model != modelName and model+':'+product == unknowns[i]:
                                        K_products = K_products * pow(thermodynamics[i],stoichiometry)
                                        break


                def func(X, r):
                    reactants = np.ones((len(t_eval)))
                    products= np.ones((len(t_eval)))


                    for x in range(0,len(reactantsSpecies)):
                        reactants = reactants * X[x]            

                    for x in range(len(reactantsSpecies),len(reactantsSpecies)+len(productsSpecies)):
                        products = products * X[x]


                    v = r*(K_reactants*reactants- K_products*products)


                    return v

                bounds=[[0],[np.inf]]
                popt, pcov = curve_fit(func,solve,Solutions[modelName][reaction], maxfev=300000, bounds=bounds)
                if popt[0]==1: 
                    popt, pcov = curve_fit(func,solve,Solutions[modelName][reaction], maxfev=300000, method='lm')

                reactionRates[modelName][reaction]=popt[0]
                
                
                
        unitsImport = ['def model composed_model as',
               '\tdef import using "units_BG.cellml" for',
                '\t\tunit concentrationUnit using unit concentrationUnit;',
               '\t\tunit speciesConstantUnit using unit speciesConstantUnit;',
               '\t\tunit fluxUnit using unit fluxUnit;',
                '\tenddef;',
              '\tdef comp main as']

        with open('./savedModels/BGCellMLmodel.txt', 'w') as fp:
            pass    
        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
            for line in unitsImport:
                file.write(line+'\n')


        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
            file.write('\t\tvar t: second {init: 0};'+'\n')


        for z,uniqueSpecies in enumerate(unknowns):
            modelName,species = uniqueSpecies.split(':')
            with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                file.write('\t\tvar ' +modelName+'_'+species+': '+'concentrationUnit '+'{'+'init: '+str([savedInitialValues[modelName][species]][0])+'}'+';'+'\n')
            with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                file.write('\t\tvar ' 'K_'+modelName+'_'+species+': '+'speciesConstantUnit '+'{'+'init: '+str(thermodynamics[z])+'}'+';'+'\n')


        for modelName,Type in savedtypes.items():
            for reaction in reactionRates[modelName]:
                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                    file.write('\t\tvar '+modelName+'_'+ reaction+': '+'fluxUnit '+';'+'\n')
                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                    file.write('\t\tvar '+'r_'+modelName+'_'+ reaction+': '+'fluxUnit '+'{'+'init: '+str(reactionRates[modelName][reaction])+'}'+';'+'\n')




        # Generating the ODEs       
        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
            file.write('\n')

        for z,uniqueSpecies in enumerate(unknowns):
            x1,x2 = uniqueSpecies.split(':')
            with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                file.write('\t\tode('+x1+'_'+x2+',t)=')
            for modelName,Type in savedtypes.items():
                for i,species in enumerate(savedSpeciesNoDuplicateSingleDict[modelName]):
                    for j,reaction in enumerate(bgTemplates[Type]['reactants'].keys()):    
                        if uniqueSpecies == modelName+':'+species:
                            if savedNmatrices[model][i][j] != 0:
                                if savedNmatrices[model][i][j] > 0:
                                    with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                                        file.write('+'+str(savedNmatrices[model][i][j])+'{dimensionless}*'+modelName+'_'+reaction)

                                if savedNmatrices[model][i][j] < 0:
                                    with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                                        file.write(str(savedNmatrices[model][i][j])+'{dimensionless}*'+modelName+'_'+reaction)

                    if species in toRemove.keys() and modelName in toRemove[species]:
                        if modelName in same_rdf[species]:
                            for model in same_rdf[species]:                            
                                if model != modelName and model+':'+species == uniqueSpecies:
                                    for j,reaction in enumerate(bgTemplates[Type]['reactants'].keys()):
                                        if savedNmatrices[model][i][j] != 0:
                                            if savedNmatrices[model][i][j] > 0:
                                                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                                                    file.write('+'+str(savedNmatrices[model][i][j])+'{dimensionless}*'+modelName+'_'+reaction)

                                            if savedNmatrices[model][i][j] < 0:
                                                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                                                    file.write(str(savedNmatrices[model][i][j])+'{dimensionless}*'+modelName+'_'+reaction)


            with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                file.write(';\n')


        # Generating the reaction rates
        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
            file.write('\n')

        for modelName,Type in savedtypes.items():
            for r,reaction in enumerate(reactionRates[modelName]):
                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                    file.write('\t\t '+ modelName+'_'+reaction+'= ') 

                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                    file.write('r_'+modelName+'_'+reaction+'*(')


                length = 0    
                for stoi,reactant in bgTemplates[Type]['reactants'][reaction]:
                    length+=1
                    if reactant in toRemove.keys() and modelName in toRemove[reactant]:
                        if modelName in same_rdf[reactant]:
                            for model in same_rdf[reactant]:      
                                for z,uniqueSpecies in enumerate(unknowns):
                                    if model != modelName and model+':'+reactant == uniqueSpecies:
                                        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                                            file.write('pow('+'K_'+model+'_'+reactant+'*'+model+'_'+reactant+','+str(stoi)+'{dimensionless}'+')')                                     
                    else:                         
                        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                            file.write('pow('+'K_'+modelName+'_'+reactant+'*'+modelName+'_'+reactant+','+str(stoi)+'{dimensionless}'+')') 

                    if length < len(bgTemplates[Type]['reactants'][reaction]):
                        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                            file.write(' * ')



                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                    file.write(' - ')


                length = 0
                for stoi,product in bgTemplates[Type]['products'][reaction]:
                    length+=1
                    if product in toRemove.keys() and modelName in toRemove[product]:
                        if modelName in same_rdf[product]:
                            for model in same_rdf[product]:      
                                for z,uniqueSpecies in enumerate(unknowns):
                                    if model != modelName and model+':'+product == uniqueSpecies:
                                        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                                            file.write('pow('+'K_'+model+'_'+product+'*'+model+'_'+product+','+str(stoi)+'{dimensionless}'+')')                                     
                    else:                         
                        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                            file.write('pow('+'K_'+modelName+'_'+product+'*'+modelName+'_'+product+','+str(stoi)+'{dimensionless}'+')') 

                    if length < len(bgTemplates[Type]['products'][reaction]):
                        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                            file.write(' * ')


                with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
                    file.write(');\n')



        with open('./savedModels/BGCellMLmodel.txt', 'a') as file:
            file.write('\tenddef;'+'\n'+'enddef;')
            
            
        text_0 = widgets.HTML(value="<h3>The CellML code for the composed bond graph model is:</h3>")
        vbox_text = widgets.VBox([text_0])
        display(vbox_text)
        with open('./savedModels/BGCellMLmodel.txt') as f:
            contents = f.read()
        print(contents)

button_compose.on_click(button_compose_clicked)
vbox_result = widgets.VBox([button_compose, output10]) 
display(vbox_result)